In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
import os
os.listdir('/content/drive/MyDrive/')
# método em Python usado para alterar o diretório de trabalho atual para o caminho especificado.
path = "/content/drive/MyDrive/Iniciacao_cientifica/"
os.chdir(path)
import shutil
shutil.copy('/content/drive/MyDrive/Iniciacao_cientifica/MLCQ_PMD_Organic_Designite_CKclass_CKmethod_Agreement_With_Agreement_Smell_Complete.csv', '/content/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/MLCQ_PMD_Organic_Designite_CKclass_CKmethod_Agreement_With_Agreement_Smell_Complete.csv'

#### Import the libraries

In [1]:
!pip install pycaret


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install optuna


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install dataframe_image


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dataframe_image as dfi
from pycaret.classification import *

In [5]:
df = pd.read_csv('./out.csv', index_col=0)
#df_type = df['Type']
df.shape

(17833, 193)

#### Remove duplicates

In [6]:
duplicate_rows = df[df.duplicated()]
len(duplicate_rows)

136

In [7]:
df.columns.values

array(['commit_hash', 'start_line', 'end_line', 'smell', 'id',
       'reviewer_id', 'sample_id', 'severity', 'longmethod_label',
       'featureenvy_label', 'dataclass_label', 'blob_label',
       'agreement_experts', 'agreement_smell_type', 'agreement_smell',
       'system_name', 'review_timestamp', 'type', 'code_name',
       'repository', 'path', 'link', 'is_from_industry_relevant_project',
       'weighted_average', 'qtd_rewier_id', 'Organic_fileRelativePath',
       'Organic_startLine', 'Organic_endLine', 'Organic_WeighOfClass',
       'Organic_NumberOfAccessorMethods',
       'Organic_ClassEffectiveLinesOfCode', 'Organic_LCOM3',
       'Organic_TightClassCohesion', 'Organic_IsAbstract',
       'Organic_PublicFieldCount', 'Organic_OverrideRatio',
       'Organic_WeightedMethodCount', 'smell_Organic',
       'Organic_smellReason', 'Organic_ChangingMethods',
       'Organic_NumberOfTryStatements', 'Organic_MaxNesting',
       'Organic_NumberOfFinallyStatements',
       'Organic_Nu

In [8]:
df = df.drop('start_line',1)
df = df.drop('end_line',1)
df = df.drop('id',1)
df = df.drop('reviewer_id',1)
df = df.drop('smell',1)
df = df.drop('sample_id',1)
df = df.drop('severity',1)
df = df.drop('review_timestamp',1)
df = df.drop('code_name', 1)
df = df.drop('repository', 1)
df = df.drop('path',1)
df = df.drop('link',1)
df = df.drop('weighted_average', 1)
df = df.drop('qtd_rewier_id', 1)
df = df.drop('is_from_industry_relevant_project',1)
df = df.drop('Organic_fileRelativePath', 1)
df = df.drop('Organic_startLine', 1)
df = df.drop('Organic_endLine', 1)
df = df.drop('PMD_Problem', 1)
df = df.drop('PMD_Package', 1)
df = df.drop('PMD_File', 1)
df = df.drop('PMD_Priority', 1)
df = df.drop('PMD_Line', 1)
#df = df.drop('PMD_Rule', 1)
df = df.drop('implementation_Project Name', 1)
df = df.drop('implementation_code_name', 1)
df = df.drop('implementation_Type Name', 1)
df = df.drop('implementation_Method Name', 1)
df = df.drop('implementation_Method start line no', 1)
df = df.drop('CK_Class_file', 1)
df = df.drop('CK_Class_class', 1)
df = df.drop('Designite_Project Name',1)
df = df.drop('Designite_Package Name',1)
df = df.drop('Unnamed: 0',1)



## Drop Code Smell - LongMethod

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the Long Method smell. To train other models, remove the other code smells.

In [9]:
#consider only the column 'longmethod_label'

df_longMethod = df.drop(columns=['featureenvy_label','dataclass_label','blob_label'])
df_longMethod.columns

Index(['commit_hash', 'longmethod_label', 'agreement_experts',
       'agreement_smell_type', 'agreement_smell', 'system_name', 'type',
       'Organic_WeighOfClass', 'Organic_NumberOfAccessorMethods',
       'Organic_ClassEffectiveLinesOfCode',
       ...
       'smell_Organic_blob', 'smell_PMD_longmethod', 'smell_PMD_featureenvy',
       'smell_PMD_dataclass', 'smell_PMD_blob', 'smell_Designite_longmethod',
       'smell_Designite_featureenvy', 'smell_Designite_dataclass',
       'smell_Designite_blob', 'background'],
      dtype='object', length=157)

#### Check the correlation and remove the high-correlated features from the dataset

In [10]:
corr = df_longMethod.corr()
threshold = 0.9
#columns = np.full((corr.shape[0],), True,dtype='object')
columns = np.isin(df_longMethod.columns,corr.index)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = df_longMethod.columns[columns]
high_corr = set(df_longMethod.columns) - set(selected_columns)
df_longMethod = df_longMethod[selected_columns]
df_longMethod.columns

Index(['longmethod_label', 'agreement_experts', 'agreement_smell_type',
       'agreement_smell', 'Organic_WeighOfClass',
       'Organic_NumberOfAccessorMethods', 'Organic_ClassEffectiveLinesOfCode',
       'Organic_LCOM3', 'Organic_TightClassCohesion',
       'Organic_PublicFieldCount', 'Organic_OverrideRatio',
       'Organic_WeightedMethodCount', 'Organic_ChangingMethods',
       'Organic_NumberOfTryStatements', 'Organic_MaxNesting',
       'Organic_NumberOfFinallyStatements', 'Organic_NumberOfThrowStatements',
       'Organic_NumberOfTryStatementsWithNoCatchAndFinally',
       'Organic_CouplingIntensity', 'Organic_NumberOfAccessedVariables',
       'Organic_MethodEffectiveLinesOfCode', 'Organic_CouplingDispersion',
       'Organic_NumberOfDummyExceptionHandlers', 'Organic_ParameterCount',
       'Organic_ChangingClasses', 'Organic_NumberOfCatchStatements',
       'Organic_ThrownExceptionTypesCount', 'CK_Class_synchronizedFieldsQty',
       'Stars', 'Watching', 'Commits', 'LOC', 'N

##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of Long Method.

In [11]:
results = df_longMethod.longmethod_label.values
df_longMethod = df_longMethod.drop('longmethod_label', axis=1)
df_longMethod['longmethod_label'] = results
df_longMethod.columns

Index(['agreement_experts', 'agreement_smell_type', 'agreement_smell',
       'Organic_WeighOfClass', 'Organic_NumberOfAccessorMethods',
       'Organic_ClassEffectiveLinesOfCode', 'Organic_LCOM3',
       'Organic_TightClassCohesion', 'Organic_PublicFieldCount',
       'Organic_OverrideRatio', 'Organic_WeightedMethodCount',
       'Organic_ChangingMethods', 'Organic_NumberOfTryStatements',
       'Organic_MaxNesting', 'Organic_NumberOfFinallyStatements',
       'Organic_NumberOfThrowStatements',
       'Organic_NumberOfTryStatementsWithNoCatchAndFinally',
       'Organic_CouplingIntensity', 'Organic_NumberOfAccessedVariables',
       'Organic_MethodEffectiveLinesOfCode', 'Organic_CouplingDispersion',
       'Organic_NumberOfDummyExceptionHandlers', 'Organic_ParameterCount',
       'Organic_ChangingClasses', 'Organic_NumberOfCatchStatements',
       'Organic_ThrownExceptionTypesCount', 'CK_Class_synchronizedFieldsQty',
       'Stars', 'Watching', 'Commits', 'LOC', 'Number of Contributor

#### Setup the model creation with pycaret

In [12]:
s = setup(data=df_longMethod, target='longmethod_label', feature_selection=True,
          remove_multicollinearity=True,
          multicollinearity_threshold=0.85,
          feature_selection_method='classic',
          fix_imbalance=True, fold=10,index=False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 26109, number of negative: 26109
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003824 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3459
[LightGBM] [Info] Number of data points in the train set: 52218, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


,Description,Value
0,Session id,8428
1,Target,longmethod_label
2,Target type,Binary
3,Original data shape,"(40951, 55)"
4,Transformed data shape,"(64504, 11)"
5,Transformed train set shape,"(52218, 11)"
6,Transformed test set shape,"(12286, 11)"
7,Numeric features,54
8,Rows with missing values,100.0%
9,Preprocess,True


In [13]:
df = get_config('X')
df

,agreement_experts,agreement_smell_type,agreement_smell,Organic_WeighOfClass,Organic_NumberOfAccessorMethods,Organic_ClassEffectiveLinesOfCode,Organic_LCOM3,Organic_TightClassCohesion,Organic_PublicFieldCount,Organic_OverrideRatio,...,smell_Organic_dataclass,smell_Organic_blob,smell_PMD_longmethod,smell_PMD_featureenvy,smell_PMD_dataclass,smell_PMD_blob,smell_Designite_longmethod,smell_Designite_featureenvy,smell_Designite_dataclass,smell_Designite_blob
0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,1.0,0.0,137.0,0.855556,0.333333,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40946,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
40947,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40948,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40949,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


##### Train the models. In this code the model was trained for the Long Method smell. From all models, we selected the top-5 in terms of F1.

In [14]:
top5_models = compare_models(n_select=5, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8763,0.9641,0.9754,0.4174,0.5846,0.5253,0.5909,0.6310
rf,Random Forest Classifier,0.8767,0.9632,0.9718,0.4180,0.5845,0.5253,0.5900,0.7520
dt,Decision Tree Classifier,0.8766,0.9630,0.9718,0.4177,0.5842,0.5250,0.5898,0.4380
xgboost,Extreme Gradient Boosting,0.8725,0.9612,0.9687,0.4095,0.5756,0.5147,0.5811,0.4990
knn,K Neighbors Classifier,0.9113,0.9208,0.6272,0.5041,0.5577,0.5092,0.5137,0.5710
lightgbm,Light Gradient Boosting Machine,0.8620,0.9581,0.9726,0.3903,0.5570,0.4924,0.5647,0.5390
gbc,Gradient Boosting Classifier,0.7595,0.9176,0.9667,0.2664,0.4176,0.3230,0.4301,1.1880
ada,Ada Boost Classifier,0.7475,0.9059,0.9405,0.2533,0.3991,0.3009,0.4052,0.7420
ridge,Ridge Classifier,0.6118,0.0000,1.0000,0.1868,0.3148,0.1936,0.3274,0.3970
lda,Linear Discriminant Analysis,0.6118,0.8485,1.0000,0.1868,0.3148,0.1936,0.3274,0.3860


In [15]:
df2=pull()
dfi.export(df2, 'LongMethod_RQ_3_2.png')

#### Tune the models

In [16]:
tuned_top5 = [
  tune_model(
    i,
    n_iter=30,
    optimize="f1",
    search_library='optuna',
    choose_better=True,
    early_stopping=True)
  for i in top5_models
  ]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7154,0.9117,0.9922,0.2378,0.3837,0.2800,0.4013
1,0.7168,0.9156,1.0000,0.2397,0.3867,0.2835,0.4064
2,0.7213,0.9111,1.0000,0.2427,0.3905,0.2883,0.4104
3,0.7178,0.9064,1.0000,0.2404,0.3876,0.2846,0.4073
4,0.7213,0.9081,0.9922,0.2417,0.3887,0.2862,0.4065
5,0.6936,0.9158,0.9922,0.2241,0.3656,0.2579,0.3826
6,0.7247,0.9094,0.9961,0.2438,0.3917,0.2902,0.4109
7,0.7163,0.9171,1.0000,0.2388,0.3855,0.2824,0.4055
8,0.7202,0.9031,0.9922,0.2403,0.3869,0.2843,0.4050


[I 2024-01-19 23:06:36,182] Searching the best hyperparameters using 28665 samples...
[I 2024-01-19 23:12:23,850] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7708,0.9375,1.0000,0.2804,0.4380,0.3469,0.4581
1,0.7701,0.9419,0.9922,0.2788,0.4353,0.3438,0.4536
2,0.7824,0.9364,0.9961,0.2904,0.4497,0.3614,0.4686
3,0.7764,0.9329,0.9961,0.2849,0.4431,0.3533,0.4622
4,0.7743,0.9257,0.9766,0.2806,0.4359,0.3451,0.4504
5,0.7788,0.9414,0.9922,0.2859,0.4439,0.3547,0.4623
6,0.7791,0.9275,0.9882,0.2857,0.4433,0.3541,0.4607
7,0.7631,0.9438,1.0000,0.2730,0.4289,0.3361,0.4495
8,0.7638,0.9251,0.9961,0.2731,0.4287,0.3360,0.4483


[I 2024-01-19 23:12:51,119] Searching the best hyperparameters using 28665 samples...
[I 2024-01-19 23:18:30,161] Finished hyperparameter search!


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8594,0.9644,0.9922,0.3878,0.5576,0.4925,0.5696
1,0.8629,0.9689,0.9961,0.3941,0.5648,0.5009,0.5771
2,0.8678,0.9610,1.0000,0.4031,0.5746,0.5126,0.5871
3,0.8626,0.9642,1.0000,0.3938,0.5651,0.5012,0.5783
4,0.8608,0.9556,0.9844,0.3895,0.5581,0.4933,0.5683
5,0.8625,0.9700,0.9961,0.3926,0.5632,0.4993,0.5758
6,0.8517,0.9572,0.9922,0.3743,0.5435,0.4758,0.5567
7,0.8608,0.9646,1.0000,0.3899,0.5611,0.4966,0.5747
8,0.8632,0.9623,1.0000,0.3941,0.5654,0.5018,0.5788


[I 2024-01-19 23:21:41,387] Searching the best hyperparameters using 28665 samples...
[I 2024-01-19 23:24:51,694] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9369,0.9613,0.7109,0.6298,0.6679,0.6331,0.6346
1,0.9334,0.9628,0.6680,0.6173,0.6417,0.6050,0.6056
2,0.9306,0.9624,0.6328,0.6067,0.6195,0.5813,0.5815
3,0.9341,0.9660,0.6133,0.6356,0.6243,0.5881,0.5882
4,0.9288,0.9581,0.6133,0.5992,0.6062,0.5671,0.5671
5,0.9403,0.9690,0.6784,0.6603,0.6692,0.6365,0.6365
6,0.9302,0.9564,0.6353,0.6022,0.6183,0.5799,0.5802
7,0.9330,0.9637,0.6353,0.6207,0.6279,0.5911,0.5912
8,0.9396,0.9655,0.7098,0.6464,0.6766,0.6434,0.6443


[I 2024-01-19 23:25:08,502] Searching the best hyperparameters using 28665 samples...
[I 2024-01-19 23:27:49,287] Finished hyperparameter search!


#### Calculing top 5 most relevant features

##### Linear Regression (LR)

In [17]:
best_model = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.4918,0.5774,0.6406,0.1073,0.1838,0.0363,0.0674
1,0.2218,0.5811,0.8789,0.0928,0.1678,0.0075,0.0287
2,0.5469,0.6040,0.6133,0.1157,0.1947,0.0523,0.0878
3,0.5445,0.5888,0.5938,0.1123,0.1888,0.0455,0.0762
4,0.4524,0.5558,0.6445,0.1004,0.1737,0.0227,0.0450
5,0.5712,0.5883,0.5098,0.1053,0.1746,0.0318,0.0500
6,0.2756,0.5843,0.8745,0.0984,0.1768,0.0201,0.0642
7,0.5611,0.6143,0.6000,0.1169,0.1957,0.0549,0.0899
8,0.2729,0.5756,0.8784,0.0984,0.1769,0.0201,0.0649


In [21]:
plot_model(best_model, plot='feature',save=True)

'Feature Importance.png'

##### Support Vector Machine (SVM)

In [22]:
best_model_svm = create_model('svm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.2020,0.0000,0.9102,0.0933,0.1692,0.0086,0.0363
1,0.1259,0.0000,0.9414,0.0882,0.1613,-0.0024,-0.0170
2,0.0900,0.0000,1.0000,0.0894,0.1640,0.0001,0.0083
3,0.1308,0.0000,0.9453,0.0890,0.1626,-0.0007,-0.0048
4,0.2020,0.0000,0.8906,0.0916,0.1662,0.0051,0.0211
5,0.0956,0.0000,1.0000,0.0896,0.1644,0.0013,0.0255
6,0.9110,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.1277,0.0000,0.9490,0.0887,0.1622,-0.0006,-0.0047
8,0.0890,0.0000,1.0000,0.0890,0.1634,0.0000,0.0000


In [24]:
plot_model(best_model_svm, plot='feature',save=True)

'Feature Importance.png'

##### Decision Tree (DT)

In [25]:
best_model_dt = create_model('dt')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8727,0.9676,0.9805,0.4108,0.5790,0.5184,0.5867
1,0.8797,0.9700,0.9883,0.4252,0.5946,0.5368,0.6028
2,0.8821,0.9637,0.9648,0.4288,0.5938,0.5364,0.5970
3,0.8797,0.9663,0.9648,0.4237,0.5888,0.5305,0.5924
4,0.8762,0.9533,0.9336,0.4142,0.5738,0.5137,0.5719
5,0.8807,0.9683,0.9804,0.4259,0.5938,0.5363,0.6006
6,0.8674,0.9567,0.9647,0.3987,0.5642,0.5014,0.5698
7,0.8831,0.9674,0.9647,0.4301,0.5949,0.5381,0.5982
8,0.8751,0.9635,0.9922,0.4154,0.5856,0.5262,0.5956


In [27]:
plot_model(best_model_dt,plot='feature',save=True)

'Feature Importance.png'

##### Random Forest (RF)

In [28]:
best_model_rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8706,0.9661,0.9844,0.4071,0.5760,0.5147,0.5848
1,0.8783,0.9675,0.9883,0.4224,0.5918,0.5334,0.6003
2,0.8811,0.9624,0.9805,0.4276,0.5955,0.5380,0.6020
3,0.8786,0.9664,0.9609,0.4212,0.5857,0.5270,0.5887
4,0.8723,0.9562,0.9648,0.4089,0.5744,0.5134,0.5791
5,0.8803,0.9678,0.9843,0.4254,0.5941,0.5365,0.6017
6,0.8667,0.9556,0.9569,0.3967,0.5609,0.4977,0.5650
7,0.8733,0.9648,0.9686,0.4103,0.5764,0.5159,0.5820
8,0.8754,0.9640,0.9843,0.4156,0.5844,0.5250,0.5928


In [30]:
plot_model(best_model_rf,plot='feature',save=True)

'Feature Importance.png'

#### Ensemble the top-5 models

In [ ]:
blended_models = blend_models(
  tuned_top5,
  choose_better=True,
  optimize="f1")

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,23:59:28
Status,. . . . . . . . . . . . . . . . . .,Compiling Final Results
Estimator,. . . . . . . . . . . . . . . . . .,Voting Classifier


In [46]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,K Neighbors Classifier,0.9355,0.9661,0.6396,0.6379,0.6387,0.6033,0.6033


#### Plot the ensemble model

In [50]:
result = pull()

In [51]:
result.to_csv('models_longmethod2.csv')

In [52]:
final_best = finalize_model(blended_models)
save_model(final_best, 'model_longmethod')

IndexError: Length of values (74598) does not match length of index (40951). This usually happens when transformations that drop rows aren't applied on all the columns.

#### Finalize and save the ensemble model

#### Interpret the ensemble model

In [43]:
!pip install shap
interpret_model(tuned_top5[0], save=True)


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the Long Method smell. To train other models, remove the other code smells.

## Drop Code Smell - FeatureEnvy

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the  Feature Envy smell. To train other models, remove the other code smells.

In [71]:
#consider only the column 'featureenvy_label'
df.columns.values
df_featureenvy = df.drop(columns=['longmethod_label','dataclass_label','blob_label'])
df_featureenvy.columns

Index(['commit_hash', 'featureenvy_label', 'agreement_experts',
       'agreement_smell_type', 'agreement_smell', 'system_name', 'type',
       'Organic_WeighOfClass', 'Organic_NumberOfAccessorMethods',
       'Organic_ClassEffectiveLinesOfCode',
       ...
       'smell_Organic_blob', 'smell_PMD_longmethod', 'smell_PMD_featureenvy',
       'smell_PMD_dataclass', 'smell_PMD_blob', 'smell_Designite_longmethod',
       'smell_Designite_featureenvy', 'smell_Designite_dataclass',
       'smell_Designite_blob', 'background'],
      dtype='object', length=157)

#### Check the correlation and remove the high-correlated features from the dataset

In [72]:
corr = df_featureenvy.corr()
threshold = 0.9
#columns = np.full((corr.shape[0],), True,dtype='object')
columns = np.isin(df_featureenvy.columns,corr.index)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = df_featureenvy.columns[columns]
high_corr = set(df_featureenvy.columns) - set(selected_columns)
df_featureenvy = df_featureenvy[selected_columns]
df_featureenvy.columns

Index(['featureenvy_label', 'agreement_experts', 'agreement_smell_type',
       'agreement_smell', 'Organic_WeighOfClass',
       'Organic_NumberOfAccessorMethods', 'Organic_ClassEffectiveLinesOfCode',
       'Organic_LCOM3', 'Organic_TightClassCohesion',
       'Organic_PublicFieldCount', 'Organic_OverrideRatio',
       'Organic_WeightedMethodCount', 'Organic_ChangingMethods',
       'Organic_NumberOfTryStatements', 'Organic_MaxNesting',
       'Organic_NumberOfFinallyStatements', 'Organic_NumberOfThrowStatements',
       'Organic_NumberOfTryStatementsWithNoCatchAndFinally',
       'Organic_CouplingIntensity', 'Organic_NumberOfAccessedVariables',
       'Organic_MethodEffectiveLinesOfCode', 'Organic_CouplingDispersion',
       'Organic_NumberOfDummyExceptionHandlers', 'Organic_ParameterCount',
       'Organic_ChangingClasses', 'Organic_NumberOfCatchStatements',
       'Organic_ThrownExceptionTypesCount', 'CK_Class_synchronizedFieldsQty',
       'Stars', 'Watching', 'Commits', 'LOC', '

##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of Long Method.

In [73]:
results = df_featureenvy.featureenvy_label.values
df_featureenvy = df_featureenvy.drop('featureenvy_label', axis=1)
df_featureenvy['featureenvy_label'] = results
df_featureenvy.columns

Index(['agreement_experts', 'agreement_smell_type', 'agreement_smell',
       'Organic_WeighOfClass', 'Organic_NumberOfAccessorMethods',
       'Organic_ClassEffectiveLinesOfCode', 'Organic_LCOM3',
       'Organic_TightClassCohesion', 'Organic_PublicFieldCount',
       'Organic_OverrideRatio', 'Organic_WeightedMethodCount',
       'Organic_ChangingMethods', 'Organic_NumberOfTryStatements',
       'Organic_MaxNesting', 'Organic_NumberOfFinallyStatements',
       'Organic_NumberOfThrowStatements',
       'Organic_NumberOfTryStatementsWithNoCatchAndFinally',
       'Organic_CouplingIntensity', 'Organic_NumberOfAccessedVariables',
       'Organic_MethodEffectiveLinesOfCode', 'Organic_CouplingDispersion',
       'Organic_NumberOfDummyExceptionHandlers', 'Organic_ParameterCount',
       'Organic_ChangingClasses', 'Organic_NumberOfCatchStatements',
       'Organic_ThrownExceptionTypesCount', 'CK_Class_synchronizedFieldsQty',
       'Stars', 'Watching', 'Commits', 'LOC', 'Number of Contributor

#### Setup the model creation with pycaret

In [74]:
s = setup(data=df_featureenvy, target='featureenvy_label', feature_selection=True,
          remove_multicollinearity=True,
          multicollinearity_threshold=0.85,
          feature_selection_method='classic',
          fix_imbalance=True, fold=10,index=False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 26088, number of negative: 26088
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002966 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1991
[LightGBM] [Info] Number of data points in the train set: 52176, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


,Description,Value
0,Session id,2641
1,Target,featureenvy_label
2,Target type,Binary
3,Original data shape,"(40951, 55)"
4,Transformed data shape,"(64462, 11)"
5,Transformed train set shape,"(52176, 11)"
6,Transformed test set shape,"(12286, 11)"
7,Numeric features,54
8,Rows with missing values,100.0%
9,Preprocess,True


In [75]:
df = get_config('X')
df

,agreement_experts,agreement_smell_type,agreement_smell,Organic_WeighOfClass,Organic_NumberOfAccessorMethods,Organic_ClassEffectiveLinesOfCode,Organic_LCOM3,Organic_TightClassCohesion,Organic_PublicFieldCount,Organic_OverrideRatio,...,smell_Organic_dataclass,smell_Organic_blob,smell_PMD_longmethod,smell_PMD_featureenvy,smell_PMD_dataclass,smell_PMD_blob,smell_Designite_longmethod,smell_Designite_featureenvy,smell_Designite_dataclass,smell_Designite_blob
0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1.0,0.0,80.0,0.0,0.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40946,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40947,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40948,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40949,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


##### Train the models. In this code the model was trained for the Long Method smell. From all models, we selected the top-5 in terms of F1.

In [76]:
top5_models = compare_models(n_select=5, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8749,0.9641,0.9825,0.4172,0.5856,0.5257,0.5929,0.6500
dt,Decision Tree Classifier,0.8749,0.9638,0.9814,0.4172,0.5854,0.5255,0.5925,0.3460
et,Extra Trees Classifier,0.8742,0.9635,0.9794,0.4156,0.5835,0.5233,0.5903,0.5460
xgboost,Extreme Gradient Boosting,0.8640,0.9604,0.9860,0.3970,0.5660,0.5022,0.5756,0.4990
knn,K Neighbors Classifier,0.9152,0.9174,0.5976,0.5360,0.5603,0.5139,0.5175,0.5120
lightgbm,Light Gradient Boosting Machine,0.8544,0.9582,0.9915,0.3812,0.5506,0.4836,0.5625,0.5480
gbc,Gradient Boosting Classifier,0.7583,0.9076,0.9713,0.2676,0.4195,0.3243,0.4323,1.0970
ada,Ada Boost Classifier,0.7386,0.8916,0.9542,0.2502,0.3964,0.2961,0.4047,0.6110
ridge,Ridge Classifier,0.6325,0.0000,0.9895,0.1957,0.3266,0.2076,0.3375,0.3280
lda,Linear Discriminant Analysis,0.6319,0.8509,0.9895,0.1956,0.3265,0.2075,0.3373,0.3360


In [77]:
df2=pull()
dfi.export(df2, 'FeatureEnvy_RQ_3_2.png')

#### Tune the models

In [78]:
tuned_top5 = [
  tune_model(
    i,
    n_iter=30,
    optimize="f1",
    search_library='optuna',
    choose_better=True,
    early_stopping=True)
  for i in top5_models
  ]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7635,0.9352,0.9961,0.2752,0.4312,0.3378,0.4498
1,0.7813,0.9341,0.9922,0.2906,0.4495,0.3605,0.4669
2,0.7921,0.9437,0.9961,0.3016,0.4631,0.3770,0.4809
3,0.7939,0.9433,0.9922,0.3030,0.4642,0.3785,0.4811
4,0.7841,0.9370,1.0000,0.2942,0.4546,0.3665,0.4737
5,0.7837,0.9304,0.9922,0.2921,0.4513,0.3631,0.4689
6,0.7875,0.9291,1.0000,0.2968,0.4577,0.3707,0.4770
7,0.7704,0.9403,0.9922,0.2799,0.4366,0.3450,0.4545
8,0.7847,0.9300,1.0000,0.2949,0.4554,0.3675,0.4744


[I 2024-01-19 17:59:34,616] Searching the best hyperparameters using 28665 samples...
[I 2024-01-19 18:04:57,994] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8273,0.9423,0.9845,0.3409,0.5065,0.4303,0.5196
1,0.8253,0.9468,0.9961,0.3395,0.5064,0.4299,0.5222
2,0.8284,0.9452,0.9961,0.3436,0.5109,0.4354,0.5265
3,0.8256,0.9470,1.0000,0.3404,0.5079,0.4315,0.5245
4,0.8294,0.9464,0.9961,0.3450,0.5125,0.4372,0.5279
5,0.8339,0.9443,0.9883,0.3494,0.5163,0.4424,0.5299
6,0.8378,0.9404,0.9961,0.3556,0.5241,0.4516,0.5390
7,0.8255,0.9430,0.9922,0.3386,0.5050,0.4285,0.5202
8,0.8318,0.9397,0.9767,0.3462,0.5112,0.4362,0.5222


[I 2024-01-19 18:05:18,939] Searching the best hyperparameters using 28665 samples...
[I 2024-01-19 18:07:10,755] Finished hyperparameter search!


...

Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8668,0.9642,1.0000,0.4031,0.5746,0.5120,0.5866
1,0.8626,0.9643,0.9961,0.3954,0.5661,0.5019,0.5779
2,0.8832,0.9696,1.0000,0.4351,0.6063,0.5499,0.6158
3,0.8737,0.9629,0.9961,0.4159,0.5868,0.5267,0.5969
4,0.8692,0.9674,1.0000,0.4076,0.5791,0.5174,0.5908
5,0.8681,0.9613,1.0000,0.4047,0.5762,0.5142,0.5883
6,0.8712,0.9607,1.0000,0.4105,0.5821,0.5212,0.5937
7,0.8758,0.9685,1.0000,0.4192,0.5908,0.5316,0.6017
8,0.8636,0.9599,1.0000,0.3975,0.5689,0.5052,0.5813


[I 2024-01-19 18:11:36,000] Searching the best hyperparameters using 28665 samples...
[I 2024-01-19 18:14:52,302] Finished hyperparameter search!


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9369,0.9650,0.6550,0.6475,0.6513,0.6165,0.6166
1,0.9400,0.9660,0.7326,0.6473,0.6873,0.6542,0.6558
2,0.9351,0.9670,0.6977,0.6250,0.6593,0.6236,0.6248
3,0.9365,0.9633,0.6744,0.6397,0.6566,0.6217,0.6219
4,0.9362,0.9667,0.6744,0.6374,0.6554,0.6202,0.6205
5,0.9288,0.9618,0.6498,0.5943,0.6208,0.5816,0.5823
6,0.9327,0.9614,0.6070,0.6290,0.6178,0.5809,0.5810
7,0.9382,0.9676,0.6809,0.6481,0.6641,0.6302,0.6304
8,0.9341,0.9581,0.6434,0.6312,0.6372,0.6010,0.6010


[I 2024-01-19 18:15:10,859] Searching the best hyperparameters using 28665 samples...
[I 2024-01-19 18:18:15,841] Finished hyperparameter search!


#### Calculing top 5 most relevant features

##### Linear Regression (LR)

In [79]:
best_model = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.3059,0.5249,0.7907,0.0953,0.1701,0.0113,0.0320
1,0.4594,0.5893,0.7636,0.1168,0.2027,0.0552,0.1121
2,0.2487,0.5660,0.8721,0.0959,0.1728,0.0127,0.0439
3,0.2299,0.5687,0.8798,0.0944,0.1705,0.0096,0.0353
4,0.5159,0.5934,0.6744,0.1177,0.2005,0.0558,0.1000
5,0.2701,0.5454,0.8405,0.0953,0.1712,0.0120,0.0382
6,0.2777,0.5513,0.8327,0.0955,0.1713,0.0124,0.0386
7,0.4780,0.6093,0.7626,0.1202,0.2076,0.0624,0.1227
8,0.2585,0.5479,0.8643,0.0964,0.1735,0.0137,0.0457


In [80]:
plot_model(best_model, plot='feature',save=True)

'Feature Importance.png'

##### Support Vector Machine (SVM)

In [81]:
best_model_svm = create_model('svm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.1005,0.0000,0.9845,0.0898,0.1646,-0.0005,-0.0062
1,0.9100,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.9100,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.0956,0.0000,0.9845,0.0893,0.1638,-0.0014,-0.0246
4,0.0900,0.0000,1.0000,0.0900,0.1651,0.0000,0.0000
5,0.2114,0.0000,0.9455,0.0976,0.1770,0.0172,0.0716
6,0.0897,0.0000,1.0000,0.0897,0.1646,0.0000,0.0000
7,0.0928,0.0000,0.9883,0.0891,0.1634,-0.0013,-0.0280
8,0.0942,0.0000,0.9961,0.0901,0.1653,0.0002,0.0046


In [82]:
plot_model(best_model_svm, plot='feature',save=True)

'Feature Importance.png'

##### Decision Tree (DT)

In [83]:
best_model_dt = create_model('dt')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8737,0.9642,0.9651,0.4136,0.5791,0.5184,0.5830
1,0.8720,0.9651,0.9884,0.4120,0.5815,0.5206,0.5904
2,0.8852,0.9690,0.9767,0.4383,0.6050,0.5490,0.6096
3,0.8804,0.9660,0.9961,0.4290,0.5998,0.5422,0.6089
4,0.8723,0.9664,0.9961,0.4132,0.5841,0.5235,0.5945
5,0.8740,0.9612,0.9611,0.4130,0.5778,0.5172,0.5812
6,0.8779,0.9610,0.9883,0.4226,0.5921,0.5335,0.6003
7,0.8796,0.9694,0.9767,0.4254,0.5927,0.5345,0.5983
8,0.8688,0.9595,0.9612,0.4039,0.5688,0.5062,0.5726


In [84]:
plot_model(best_model_dt, plot='feature',save=True)

'Feature Importance.png'

##### Random Forest (RF)

In [85]:
best_model_rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8737,0.9648,0.9651,0.4136,0.5791,0.5184,0.5830
1,0.8678,0.9663,0.9922,0.4044,0.5746,0.5123,0.5849
2,0.8842,0.9670,0.9884,0.4366,0.6057,0.5495,0.6127
3,0.8814,0.9665,0.9961,0.4312,0.6019,0.5447,0.6108
4,0.8709,0.9673,0.9884,0.4100,0.5795,0.5183,0.5886
5,0.8761,0.9629,0.9844,0.4189,0.5877,0.5283,0.5954
6,0.8779,0.9605,0.9883,0.4226,0.5921,0.5335,0.6003
7,0.8789,0.9693,0.9844,0.4245,0.5932,0.5349,0.6005
8,0.8699,0.9598,0.9767,0.4071,0.5747,0.5128,0.5815


In [86]:
plot_model(best_model_rf, plot='feature',save=True)

'Feature Importance.png'

#### Ensemble the top-5 models

In [88]:
blended_models = blend_models(
  tuned_top5,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8713,0.9662,0.9884,0.4106,0.5802,0.5191,0.5892
1,0.8671,0.9683,0.9922,0.4031,0.5733,0.5107,0.5837
2,0.8842,0.9696,1.0000,0.4373,0.6085,0.5524,0.6178
3,0.8783,0.9673,0.9961,0.4248,0.5956,0.5372,0.6051
4,0.8737,0.9686,0.9961,0.4159,0.5868,0.5267,0.5969
5,0.8758,0.9647,0.9805,0.4179,0.5860,0.5265,0.5930
6,0.8779,0.9623,0.9883,0.4226,0.5921,0.5335,0.6003
7,0.8765,0.9696,0.9961,0.4204,0.5912,0.5322,0.6012
8,0.8699,0.9608,0.9767,0.4071,0.5747,0.5128,0.5815


Original model was better than the blended model, hence it will be returned. NOTE: The display metrics are for the blended model (not the original one).


In [89]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,K Neighbors Classifier,0.9344,0.9622,0.6594,0.6287,0.6437,0.6076,0.6078


In [90]:
result = pull()

In [91]:
result.to_csv('models_featureenvy.csv')

#### Interpret the ensemble model

In [92]:
final_best = finalize_model(blended_models)
save_model(final_best, 'model_featureenvy')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['agreement_experts',
                                              'agreement_smell_type',
                                              'agreement_smell',
                                              'Organic_WeighOfClass',
                                              'Organic_NumberOfAccessorMethods',
                                              'Organic_ClassEffectiveLinesOfCode',
                                              'Organic_LCOM3',
                                              'Organic_TightClassCohesion',
                                              'Organic_PublicFieldCount',
                                              'Organic_Override...
                                                                 norm_order=1,
                                                                 prefit=Fal

#### Finalize and save the ensemble model

In [93]:
!pip install shap
interpret_model(tuned_top5[0], save=True)


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Drop Code Smell - DataClass

In [64]:
##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the DataClass smell. To train other models, remove the other code smells.

In [41]:
#consider only the column 'dataclass_label'

df_dataclass = df.drop(columns=['longmethod_label','featureenvy_label','blob_label'])
df_dataclass.columns

Index(['commit_hash', 'dataclass_label', 'agreement_experts',
       'agreement_smell_type', 'agreement_smell', 'system_name', 'type',
       'Organic_WeighOfClass', 'Organic_NumberOfAccessorMethods',
       'Organic_ClassEffectiveLinesOfCode',
       ...
       'smell_Organic_blob', 'smell_PMD_longmethod', 'smell_PMD_featureenvy',
       'smell_PMD_dataclass', 'smell_PMD_blob', 'smell_Designite_longmethod',
       'smell_Designite_featureenvy', 'smell_Designite_dataclass',
       'smell_Designite_blob', 'background'],
      dtype='object', length=157)

#### Check the correlation and remove the high-correlated features from the dataset

In [42]:
corr = df_dataclass.corr()
threshold = 0.9
#columns = np.full((corr.shape[0],), True,dtype='object')
columns = np.isin(df_dataclass.columns,corr.index)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = df_dataclass.columns[columns]
high_corr = set(df_dataclass.columns) - set(selected_columns)
df_dataclass = df_dataclass[selected_columns]
df_dataclass.columns

Index(['dataclass_label', 'agreement_experts', 'agreement_smell_type',
       'agreement_smell', 'Organic_WeighOfClass',
       'Organic_NumberOfAccessorMethods', 'Organic_ClassEffectiveLinesOfCode',
       'Organic_LCOM3', 'Organic_TightClassCohesion',
       'Organic_PublicFieldCount', 'Organic_OverrideRatio',
       'Organic_WeightedMethodCount', 'Organic_ChangingMethods',
       'Organic_NumberOfTryStatements', 'Organic_MaxNesting',
       'Organic_NumberOfFinallyStatements', 'Organic_NumberOfThrowStatements',
       'Organic_NumberOfTryStatementsWithNoCatchAndFinally',
       'Organic_CouplingIntensity', 'Organic_NumberOfAccessedVariables',
       'Organic_MethodEffectiveLinesOfCode', 'Organic_CouplingDispersion',
       'Organic_NumberOfDummyExceptionHandlers', 'Organic_ParameterCount',
       'Organic_ChangingClasses', 'Organic_NumberOfCatchStatements',
       'Organic_ThrownExceptionTypesCount', 'CK_Class_synchronizedFieldsQty',
       'Stars', 'Watching', 'Commits', 'LOC', 'Nu

##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of DataClass.

In [43]:
results = df_dataclass.dataclass_label.values
df_dataclass = df_dataclass.drop('dataclass_label', axis=1)
df_dataclass['dataclass_label'] = results
df_dataclass.columns

Index(['agreement_experts', 'agreement_smell_type', 'agreement_smell',
       'Organic_WeighOfClass', 'Organic_NumberOfAccessorMethods',
       'Organic_ClassEffectiveLinesOfCode', 'Organic_LCOM3',
       'Organic_TightClassCohesion', 'Organic_PublicFieldCount',
       'Organic_OverrideRatio', 'Organic_WeightedMethodCount',
       'Organic_ChangingMethods', 'Organic_NumberOfTryStatements',
       'Organic_MaxNesting', 'Organic_NumberOfFinallyStatements',
       'Organic_NumberOfThrowStatements',
       'Organic_NumberOfTryStatementsWithNoCatchAndFinally',
       'Organic_CouplingIntensity', 'Organic_NumberOfAccessedVariables',
       'Organic_MethodEffectiveLinesOfCode', 'Organic_CouplingDispersion',
       'Organic_NumberOfDummyExceptionHandlers', 'Organic_ParameterCount',
       'Organic_ChangingClasses', 'Organic_NumberOfCatchStatements',
       'Organic_ThrownExceptionTypesCount', 'CK_Class_synchronizedFieldsQty',
       'Stars', 'Watching', 'Commits', 'LOC', 'Number of Contributor

#### Setup the model creation with pycaret

In [44]:
s = setup(data=df_dataclass, target='dataclass_label', feature_selection=True,
          remove_multicollinearity=True,
          multicollinearity_threshold=0.85,
          feature_selection_method='classic',
          fix_imbalance=True, fold=10,index=False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 23277, number of negative: 23277
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002661 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1815
[LightGBM] [Info] Number of data points in the train set: 46554, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


,Description,Value
0,Session id,1758
1,Target,dataclass_label
2,Target type,Binary
3,Original data shape,"(40951, 55)"
4,Transformed data shape,"(58840, 11)"
5,Transformed train set shape,"(46554, 11)"
6,Transformed test set shape,"(12286, 11)"
7,Numeric features,54
8,Rows with missing values,100.0%
9,Preprocess,True


In [45]:
df = get_config('X')
df

,agreement_experts,agreement_smell_type,agreement_smell,Organic_WeighOfClass,Organic_NumberOfAccessorMethods,Organic_ClassEffectiveLinesOfCode,Organic_LCOM3,Organic_TightClassCohesion,Organic_PublicFieldCount,Organic_OverrideRatio,...,smell_Organic_dataclass,smell_Organic_blob,smell_PMD_longmethod,smell_PMD_featureenvy,smell_PMD_dataclass,smell_PMD_blob,smell_Designite_longmethod,smell_Designite_featureenvy,smell_Designite_dataclass,smell_Designite_blob
0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0.208333,19.0,123.0,0.907407,0.695652,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40946,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40947,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40948,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40949,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


##### Train the models. In this code the model was trained for the Long Method smell. From all models, we selected the top-5 in terms of F1.

In [46]:
top5_models = compare_models(n_select=5, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8667,0.9635,0.9718,0.5883,0.7328,0.6511,0.6875,0.6210
dt,Decision Tree Classifier,0.8660,0.9634,0.9710,0.5875,0.7319,0.6498,0.6863,0.3590
et,Extra Trees Classifier,0.8645,0.9624,0.9723,0.5845,0.7299,0.6469,0.6843,0.5370
xgboost,Extreme Gradient Boosting,0.8559,0.9596,0.9670,0.5687,0.7162,0.6281,0.6679,0.4430
knn,K Neighbors Classifier,0.8948,0.9472,0.6869,0.7367,0.7100,0.6459,0.6471,0.5190
lightgbm,Light Gradient Boosting Machine,0.8406,0.9558,0.9696,0.5431,0.6960,0.5994,0.6458,0.4630
gbc,Gradient Boosting Classifier,0.7218,0.9110,0.9807,0.4018,0.5700,0.4136,0.5039,0.9120
ada,Ada Boost Classifier,0.7150,0.8914,0.9614,0.3943,0.5592,0.3989,0.4857,0.5730
ridge,Ridge Classifier,0.6648,0.0000,0.9985,0.3595,0.5286,0.3484,0.4587,0.3420
lda,Linear Discriminant Analysis,0.6558,0.8496,0.9972,0.3531,0.5215,0.3376,0.4495,0.3420


In [47]:
df2=pull()
dfi.export(df2, 'DataClass_RQ_3_2.png')

#### Tune the models

In [48]:
tuned_top5 = [
  tune_model(
    i,
    n_iter=30,
    optimize="f1",
    search_library='optuna',
    choose_better=True,
    early_stopping=True)
  for i in top5_models
  ]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7782,0.9309,0.9703,0.4576,0.6219,0.4921,0.5618
1,0.7653,0.9355,0.9703,0.4432,0.6085,0.4723,0.5463
2,0.7597,0.9330,0.9870,0.4382,0.6070,0.4686,0.5488
3,0.7653,0.9418,0.9852,0.4440,0.6121,0.4764,0.5543
4,0.7740,0.9245,0.9740,0.4530,0.6184,0.4866,0.5587
5,0.7624,0.9302,0.9703,0.4398,0.6052,0.4677,0.5427
6,0.7694,0.9227,0.9870,0.4481,0.6164,0.4828,0.5599
7,0.7711,0.9385,0.9889,0.4505,0.6190,0.4863,0.5632
8,0.7725,0.9269,0.9889,0.4521,0.6205,0.4884,0.5648


[I 2024-01-19 17:27:54,641] Searching the best hyperparameters using 28665 samples...
[I 2024-01-19 17:33:56,687] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8036,0.9324,0.9777,0.4889,0.6518,0.5354,0.5978
1,0.8158,0.9420,0.9555,0.5054,0.6611,0.5506,0.6032
2,0.7921,0.9235,0.9443,0.4735,0.6307,0.5074,0.5659
3,0.8444,0.9599,0.9592,0.5494,0.6986,0.6040,0.6466
4,0.8232,0.9436,0.9573,0.5160,0.6706,0.5641,0.6144
5,0.8276,0.9530,0.9777,0.5218,0.6805,0.5769,0.6302
6,0.8032,0.9256,0.9480,0.4876,0.6439,0.5266,0.5821
7,0.7969,0.9324,0.9518,0.4799,0.6381,0.5174,0.5760
8,0.7966,0.9293,0.9518,0.4794,0.6377,0.5168,0.5755


[I 2024-01-19 17:34:22,730] Searching the best hyperparameters using 28665 samples...
[I 2024-01-19 17:36:12,122] Finished hyperparameter search!


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8437,0.9605,1.0000,0.5461,0.7064,0.6121,0.6641
1,0.8612,0.9731,1.0000,0.5752,0.7304,0.6458,0.6906
2,0.8556,0.9698,1.0000,0.5656,0.7225,0.6348,0.6819
3,0.8455,0.9628,0.9981,0.5490,0.7084,0.6150,0.6658
4,0.8462,0.9558,1.0000,0.5500,0.7097,0.6167,0.6677
5,0.8304,0.9569,1.0000,0.5254,0.6889,0.5873,0.6448
6,0.8629,0.9700,1.0000,0.5779,0.7325,0.6489,0.6931
7,0.8569,0.9704,1.0000,0.5680,0.7245,0.6375,0.6840
8,0.8507,0.9703,1.0000,0.5574,0.7158,0.6253,0.6744


[I 2024-01-19 17:41:25,503] Searching the best hyperparameters using 28665 samples...
[I 2024-01-19 17:44:22,903] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9118,0.9700,0.8200,0.7391,0.7775,0.7226,0.7241
1,0.9142,0.9727,0.8275,0.7446,0.7838,0.7305,0.7321
2,0.9072,0.9673,0.8256,0.7212,0.7699,0.7121,0.7147
3,0.9016,0.9629,0.8126,0.7076,0.7565,0.6952,0.6978
4,0.8866,0.9539,0.7551,0.6783,0.7147,0.6442,0.6456
5,0.9058,0.9657,0.8271,0.7154,0.7672,0.7086,0.7115
6,0.8904,0.9582,0.7844,0.6806,0.7288,0.6606,0.6632
7,0.9082,0.9692,0.8386,0.7197,0.7746,0.7174,0.7207
8,0.9047,0.9679,0.8071,0.7202,0.7612,0.7019,0.7037


[I 2024-01-19 17:44:39,866] Searching the best hyperparameters using 28665 samples...
[I 2024-01-19 17:47:00,091] Finished hyperparameter search!


#### Calculing top 5 most relevant features

##### Linear Regression (LR)

In [49]:
best_model = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6104,0.5903,0.5046,0.2424,0.3275,0.0986,0.1117
1,0.5783,0.5756,0.4954,0.2218,0.3064,0.0630,0.0735
2,0.5870,0.6023,0.5436,0.2380,0.3311,0.0942,0.1110
3,0.5947,0.5938,0.5269,0.2385,0.3283,0.0937,0.1089
4,0.6167,0.5862,0.4991,0.2450,0.3286,0.1022,0.1149
5,0.6012,0.5982,0.5204,0.2403,0.3288,0.0969,0.1115
6,0.6336,0.5743,0.4294,0.2372,0.3056,0.0840,0.0909
7,0.6099,0.5834,0.4490,0.2277,0.3021,0.0700,0.0778
8,0.6786,0.5765,0.3655,0.2539,0.2996,0.0998,0.1026


In [50]:
plot_model(best_model, plot='feature',save=True)

'Feature Importance.png'

##### Support Vector Machine (SVM)

In [51]:
best_model_svm = create_model('svm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5964,0.0000,0.4286,0.2139,0.2854,0.0461,0.0515
1,0.1880,0.0000,1.0000,0.1880,0.3165,0.0000,0.0000
2,0.8116,0.0000,0.0000,0.0000,0.0000,-0.0007,-0.0090
3,0.1880,0.0000,1.0000,0.1880,0.3165,0.0000,0.0000
4,0.7771,0.0000,0.0371,0.1429,0.0589,-0.0202,-0.0262
5,0.1877,0.0000,1.0000,0.1877,0.3161,0.0000,0.0000
6,0.1877,0.0000,1.0000,0.1877,0.3161,0.0000,0.0000
7,0.1881,0.0000,1.0000,0.1881,0.3166,0.0000,0.0000
8,0.1881,0.0000,1.0000,0.1881,0.3166,0.0000,0.0000


In [52]:
plot_model(best_model_svm, plot='feature',save=True)

'Feature Importance.png'

##### Decision Tree (DT)

In [53]:
best_model_dt = create_model('dt')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8661,0.9618,0.9703,0.5870,0.7315,0.6493,0.6857
1,0.8741,0.9729,0.9796,0.6014,0.7452,0.6679,0.7029
2,0.8570,0.9595,0.9740,0.5700,0.7192,0.6319,0.6727
3,0.8587,0.9608,0.9629,0.5741,0.7193,0.6329,0.6706
4,0.8636,0.9560,0.9481,0.5847,0.7233,0.6394,0.6723
5,0.8503,0.9576,0.9758,0.5579,0.7099,0.6189,0.6628
6,0.8726,0.9617,0.9647,0.6000,0.7398,0.6615,0.6942
7,0.8747,0.9700,0.9777,0.6030,0.7459,0.6689,0.7033
8,0.8692,0.9692,0.9777,0.5921,0.7376,0.6573,0.6940


In [54]:
plot_model(best_model_dt, plot='feature',save=True)

'Feature Importance.png'

##### Random Forest (RF)

In [55]:
best_model_rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8650,0.9614,0.9722,0.5848,0.7303,0.6476,0.6848
1,0.8825,0.9729,0.9685,0.6200,0.7560,0.6834,0.7129
2,0.8549,0.9599,0.9759,0.5662,0.7166,0.6281,0.6702
3,0.8654,0.9617,0.9703,0.5857,0.7304,0.6479,0.6846
4,0.8577,0.9544,0.9573,0.5727,0.7167,0.6295,0.6665
5,0.8604,0.9677,0.9740,0.5758,0.7238,0.6385,0.6779
6,0.8723,0.9617,0.9758,0.5979,0.7415,0.6631,0.6982
7,0.8772,0.9701,0.9722,0.6086,0.7486,0.6729,0.7052
8,0.8576,0.9613,0.9740,0.5713,0.7202,0.6332,0.6737


In [56]:
plot_model(best_model_rf, plot='feature',save=True)

'Feature Importance.png'

#### Ensemble the top-5 models

In [57]:
blended_models = blend_models(
  tuned_top5,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8675,0.9627,0.9481,0.5921,0.7290,0.6473,0.6787
1,0.8730,0.9652,0.9666,0.6009,0.7411,0.6630,0.6958
2,0.8804,0.9693,0.9685,0.6156,0.7527,0.6789,0.7092
3,0.8852,0.9760,0.9666,0.6262,0.7600,0.6891,0.7171
4,0.8696,0.9580,0.9239,0.5993,0.7270,0.6463,0.6725
5,0.8709,0.9702,0.9703,0.5959,0.7383,0.6590,0.6935
6,0.8873,0.9699,0.9535,0.6326,0.7606,0.6908,0.7156
7,0.8824,0.9716,0.9573,0.6217,0.7538,0.6811,0.7085
8,0.8639,0.9620,0.9610,0.5840,0.7265,0.6430,0.6783


Original model was better than the blended model, hence it will be returned. NOTE: The display metrics are for the blended model (not the original one).


In [58]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,K Neighbors Classifier,0.9086,0.9688,0.8100,0.7323,0.7692,0.7124,0.7138


In [59]:
result = pull()

#### Plot the ensemble model

In [60]:
result.to_csv('models_dataclass.csv')

#### Finalize and save the ensemble model

In [61]:
final_best = finalize_model(blended_models)
save_model(final_best, 'model_dataclass')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['agreement_experts',
                                              'agreement_smell_type',
                                              'agreement_smell',
                                              'Organic_WeighOfClass',
                                              'Organic_NumberOfAccessorMethods',
                                              'Organic_ClassEffectiveLinesOfCode',
                                              'Organic_LCOM3',
                                              'Organic_TightClassCohesion',
                                              'Organic_PublicFieldCount',
                                              'Organic_Override...
                                                                 norm_order=1,
                                                                 prefit=Fal

#### Interpret the ensemble model

In [62]:
!pip install shap
interpret_model(tuned_top5[0], save=True)


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Drop Code Smell - Blob

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the Blob. To train other models, remove the other code smells.

In [9]:
#consider only the column 'blob_label'

df_blob = df.drop(columns=['featureenvy_label','dataclass_label','longmethod_label'])
df_blob.columns

Index(['commit_hash', 'blob_label', 'agreement_experts',
       'agreement_smell_type', 'agreement_smell', 'system_name', 'type',
       'Organic_WeighOfClass', 'Organic_NumberOfAccessorMethods',
       'Organic_ClassEffectiveLinesOfCode',
       ...
       'smell_Organic_blob', 'smell_PMD_longmethod', 'smell_PMD_featureenvy',
       'smell_PMD_dataclass', 'smell_PMD_blob', 'smell_Designite_longmethod',
       'smell_Designite_featureenvy', 'smell_Designite_dataclass',
       'smell_Designite_blob', 'background'],
      dtype='object', length=157)

#### Check the correlation and remove the high-correlated features from the dataset

In [10]:
corr = df_blob.corr()
threshold = 0.9
#columns = np.full((corr.shape[0],), True,dtype='object')
columns = np.isin(df_blob.columns,corr.index)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = df_blob.columns[columns]
high_corr = set(df_blob.columns) - set(selected_columns)
df_blob = df_blob[selected_columns]
df_blob.columns

Index(['blob_label', 'agreement_experts', 'agreement_smell_type',
       'agreement_smell', 'Organic_WeighOfClass',
       'Organic_NumberOfAccessorMethods', 'Organic_ClassEffectiveLinesOfCode',
       'Organic_LCOM3', 'Organic_TightClassCohesion',
       'Organic_PublicFieldCount', 'Organic_OverrideRatio',
       'Organic_WeightedMethodCount', 'Organic_ChangingMethods',
       'Organic_NumberOfTryStatements', 'Organic_MaxNesting',
       'Organic_NumberOfFinallyStatements', 'Organic_NumberOfThrowStatements',
       'Organic_NumberOfTryStatementsWithNoCatchAndFinally',
       'Organic_CouplingIntensity', 'Organic_NumberOfAccessedVariables',
       'Organic_MethodEffectiveLinesOfCode', 'Organic_CouplingDispersion',
       'Organic_NumberOfDummyExceptionHandlers', 'Organic_ParameterCount',
       'Organic_ChangingClasses', 'Organic_NumberOfCatchStatements',
       'Organic_ThrownExceptionTypesCount', 'CK_Class_synchronizedFieldsQty',
       'Stars', 'Watching', 'Commits', 'LOC', 'Number 

##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of Blob.

In [11]:
results = df_blob.blob_label.values
df_blob = df_blob.drop('blob_label', axis=1)
df_blob['blob_label'] = results
df_blob.columns

Index(['agreement_experts', 'agreement_smell_type', 'agreement_smell',
       'Organic_WeighOfClass', 'Organic_NumberOfAccessorMethods',
       'Organic_ClassEffectiveLinesOfCode', 'Organic_LCOM3',
       'Organic_TightClassCohesion', 'Organic_PublicFieldCount',
       'Organic_OverrideRatio', 'Organic_WeightedMethodCount',
       'Organic_ChangingMethods', 'Organic_NumberOfTryStatements',
       'Organic_MaxNesting', 'Organic_NumberOfFinallyStatements',
       'Organic_NumberOfThrowStatements',
       'Organic_NumberOfTryStatementsWithNoCatchAndFinally',
       'Organic_CouplingIntensity', 'Organic_NumberOfAccessedVariables',
       'Organic_MethodEffectiveLinesOfCode', 'Organic_CouplingDispersion',
       'Organic_NumberOfDummyExceptionHandlers', 'Organic_ParameterCount',
       'Organic_ChangingClasses', 'Organic_NumberOfCatchStatements',
       'Organic_ThrownExceptionTypesCount', 'CK_Class_synchronizedFieldsQty',
       'Stars', 'Watching', 'Commits', 'LOC', 'Number of Contributor

#### Setup the model creation with pycaret

In [12]:
s = setup(data=df_blob, target='blob_label', feature_selection=True,
          remove_multicollinearity=True,
          multicollinearity_threshold=0.85,
          feature_selection_method='classic',
          fix_imbalance=True, fold=10,index=False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 23652, number of negative: 23652
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005323 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2479
[LightGBM] [Info] Number of data points in the train set: 47304, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


,Description,Value
0,Session id,1985
1,Target,blob_label
2,Target type,Binary
3,Original data shape,"(40951, 55)"
4,Transformed data shape,"(59590, 11)"
5,Transformed train set shape,"(47304, 11)"
6,Transformed test set shape,"(12286, 11)"
7,Numeric features,54
8,Rows with missing values,100.0%
9,Preprocess,True


In [13]:
df = get_config('X')
df

,agreement_experts,agreement_smell_type,agreement_smell,Organic_WeighOfClass,Organic_NumberOfAccessorMethods,Organic_ClassEffectiveLinesOfCode,Organic_LCOM3,Organic_TightClassCohesion,Organic_PublicFieldCount,Organic_OverrideRatio,...,smell_Organic_dataclass,smell_Organic_blob,smell_PMD_longmethod,smell_PMD_featureenvy,smell_PMD_dataclass,smell_PMD_blob,smell_Designite_longmethod,smell_Designite_featureenvy,smell_Designite_dataclass,smell_Designite_blob
0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40946,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40947,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40948,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40949,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


##### Train the models. In this code the model was trained for the Long Method smell. From all models, we selected the top-5 in terms of F1.

In [15]:
top5_models = compare_models(n_select=5, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8808,0.9684,0.9745,0.5979,0.7410,0.6693,0.7030,0.8670
dt,Decision Tree Classifier,0.8806,0.9685,0.9749,0.5974,0.7407,0.6689,0.7028,0.4740
et,Extra Trees Classifier,0.8803,0.9684,0.9725,0.5972,0.7399,0.6679,0.7014,0.7670
xgboost,Extreme Gradient Boosting,0.8707,0.9644,0.9677,0.5782,0.7238,0.6463,0.6829,0.5640
lightgbm,Light Gradient Boosting Machine,0.8602,0.9607,0.9707,0.5580,0.7085,0.6252,0.6668,0.6760
knn,K Neighbors Classifier,0.8808,0.9414,0.7602,0.6375,0.6912,0.6183,0.6236,0.6130
gbc,Gradient Boosting Classifier,0.7520,0.9185,0.9729,0.4118,0.5786,0.4412,0.5232,1.5440
ada,Ada Boost Classifier,0.7360,0.9027,0.9589,0.3952,0.5597,0.4147,0.4979,0.8650
ridge,Ridge Classifier,0.6521,0.0000,1.0000,0.3346,0.5014,0.3243,0.4399,0.4550
lda,Linear Discriminant Analysis,0.6521,0.8643,1.0000,0.3346,0.5014,0.3243,0.4399,0.4700


In [16]:
df2=pull()
dfi.export(df2, 'Blob_RQ_3_2.png')

#### Tune the models

In [17]:
tuned_top5= [
  tune_model(
    i,
    n_iter=30,
    optimize="f1",
    search_library='optuna',
    choose_better=True,
    early_stopping=True)
  for i in top5_models
  ]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8319,0.9514,0.9860,0.5098,0.6721,0.5740,0.6304
1,0.8155,0.9427,0.9581,0.4858,0.6447,0.5375,0.5940
2,0.8141,0.9383,0.9761,0.4847,0.6477,0.5401,0.6012
3,0.8120,0.9440,0.9761,0.4818,0.6452,0.5365,0.5984
4,0.8123,0.9462,0.9801,0.4824,0.6465,0.5381,0.6008
5,0.8259,0.9516,0.9741,0.5010,0.6617,0.5601,0.6163
6,0.7994,0.9441,0.9701,0.4646,0.6283,0.5132,0.5785
7,0.8144,0.9467,0.9800,0.4847,0.6486,0.5413,0.6033
8,0.8179,0.9438,0.9820,0.4896,0.6534,0.5479,0.6090


[I 2024-01-19 16:36:14,196] Searching the best hyperparameters using 28665 samples...
[I 2024-01-19 16:42:50,459] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8483,0.9554,0.9820,0.5359,0.6934,0.6039,0.6528
1,0.8612,0.9563,0.9541,0.5604,0.7061,0.6231,0.6609
2,0.8396,0.9537,0.9681,0.5226,0.6788,0.5842,0.6335
3,0.8507,0.9532,0.9602,0.5416,0.6925,0.6038,0.6470
4,0.8392,0.9574,0.9681,0.5220,0.6783,0.5835,0.6330
5,0.8486,0.9546,0.9561,0.5376,0.6882,0.5983,0.6416
6,0.8346,0.9529,0.9641,0.5144,0.6708,0.5736,0.6241
7,0.8388,0.9564,0.9800,0.5207,0.6801,0.5854,0.6377
8,0.8374,0.9498,0.9741,0.5186,0.6768,0.5813,0.6329


[I 2024-01-19 16:43:22,844] Searching the best hyperparameters using 28665 samples...
[I 2024-01-19 16:45:20,025] Finished hyperparameter search!


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8713,0.9666,0.9960,0.5762,0.7301,0.6533,0.6955
1,0.8549,0.9651,0.9980,0.5464,0.7062,0.6205,0.6701
2,0.8580,0.9684,1.0000,0.5523,0.7116,0.6275,0.6762
3,0.8678,0.9680,0.9980,0.5700,0.7256,0.6468,0.6909
4,0.8629,0.9687,1.0000,0.5609,0.7187,0.6373,0.6839
5,0.8615,0.9711,1.0000,0.5579,0.7162,0.6341,0.6814
6,0.8535,0.9685,0.9960,0.5442,0.7038,0.6173,0.6671
7,0.8545,0.9661,1.0000,0.5458,0.7061,0.6202,0.6705
8,0.8454,0.9600,0.9960,0.5309,0.6926,0.6017,0.6549


[I 2024-01-19 16:50:42,737] Searching the best hyperparameters using 28665 samples...
[I 2024-01-19 16:53:47,571] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8894,0.9695,0.9741,0.6162,0.7548,0.6881,0.7181
1,0.8765,0.9653,0.9721,0.5889,0.7334,0.6593,0.6943
2,0.8772,0.9693,0.9781,0.5901,0.7361,0.6624,0.6983
3,0.8845,0.9680,0.9681,0.6067,0.7460,0.6763,0.7072
4,0.8825,0.9684,0.9721,0.6017,0.7433,0.6725,0.7050
5,0.8814,0.9709,0.9701,0.5993,0.7409,0.6694,0.7021
6,0.8737,0.9696,0.9721,0.5832,0.7290,0.6533,0.6895
7,0.8733,0.9664,0.9721,0.5825,0.7285,0.6525,0.6889
8,0.8723,0.9658,0.9721,0.5805,0.7269,0.6503,0.6871


[I 2024-01-19 16:54:03,629] Searching the best hyperparameters using 28665 samples...
[I 2024-01-19 17:00:49,824] Finished hyperparameter search!


#### Calculing top 5 most relevant features

##### Linear Regression (LR)

In [18]:
best_model = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5270,0.5664,0.5489,0.1957,0.2886,0.0417,0.0542
1,0.6613,0.5988,0.4651,0.2489,0.3243,0.1251,0.1360
2,0.3397,0.5122,0.7510,0.1758,0.2849,0.0015,0.0030
3,0.5734,0.6045,0.5538,0.2177,0.3125,0.0817,0.1005
4,0.6760,0.6216,0.5040,0.2712,0.3526,0.1618,0.1756
5,0.5921,0.5929,0.5429,0.2244,0.3176,0.0933,0.1118
6,0.5394,0.6119,0.6068,0.2130,0.3154,0.0763,0.1002
7,0.6347,0.6097,0.5389,0.2486,0.3403,0.1328,0.1518
8,0.5290,0.5960,0.6028,0.2078,0.3091,0.0664,0.0882


In [19]:
plot_model(best_model, plot='feature',save=True)

'Feature Importance.png'

##### Support Vector Machine (SVM)

In [20]:
best_model_svm = create_model('svm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8253,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.1747,0.0000,1.0000,0.1747,0.2975,0.0000,0.0000
2,0.1758,0.0000,1.0000,0.1752,0.2982,0.0003,0.0122
3,0.1751,0.0000,1.0000,0.1751,0.2980,0.0000,0.0000
4,0.1751,0.0000,1.0000,0.1751,0.2980,0.0000,0.0000
5,0.1874,0.0000,0.9681,0.1733,0.2940,-0.0035,-0.0248
6,0.1752,0.0000,0.9860,0.1733,0.2947,-0.0037,-0.0557
7,0.8224,0.0000,0.0180,0.3462,0.0342,0.0172,0.0432
8,0.1881,0.0000,0.9661,0.1732,0.2938,-0.0038,-0.0259


In [21]:
plot_model(best_model_svm, plot='feature',save=True)

'Feature Importance.png'

##### Decision Tree (DT)

In [24]:
best_model_dt = create_model('dt')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8880,0.9682,0.9780,0.6125,0.7533,0.6857,0.7171
1,0.8668,0.9576,0.9721,0.5696,0.7183,0.6387,0.6778
2,0.8800,0.9693,0.9741,0.5963,0.7398,0.6676,0.7015
3,0.8877,0.9690,0.9661,0.6139,0.7508,0.6829,0.7121
4,0.8884,0.9685,0.9661,0.6155,0.7519,0.6844,0.7134
5,0.8838,0.9727,0.9800,0.6032,0.7468,0.6768,0.7104
6,0.8737,0.9683,0.9701,0.5834,0.7286,0.6528,0.6887
7,0.8709,0.9676,0.9741,0.5775,0.7251,0.6478,0.6856
8,0.8716,0.9676,0.9800,0.5783,0.7274,0.6506,0.6893


In [23]:
plot_model(best_model_dt, plot='feature',save=True)

'Feature Importance.png'

##### Random Forest (RF)

In [25]:
best_model_rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8936,0.9700,0.9760,0.6253,0.7623,0.6979,0.7266
1,0.8790,0.9654,0.9721,0.5939,0.7373,0.6645,0.6986
2,0.8786,0.9698,0.9761,0.5932,0.7380,0.6650,0.6999
3,0.8852,0.9674,0.9701,0.6080,0.7475,0.6782,0.7092
4,0.8839,0.9692,0.9761,0.6042,0.7464,0.6764,0.7091
5,0.8814,0.9715,0.9820,0.5978,0.7432,0.6719,0.7069
6,0.8761,0.9687,0.9541,0.5901,0.7292,0.6546,0.6864
7,0.8716,0.9667,0.9780,0.5785,0.7270,0.6501,0.6885
8,0.8754,0.9670,0.9880,0.5851,0.7350,0.6604,0.6991


In [26]:
plot_model(best_model_rf, plot='feature',save=True)

'Feature Importance.png'

#### Ensemble the top-5 models

In [27]:
blended_models = blend_models(
  top5_models,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8887,0.9688,0.9800,0.6138,0.7548,0.6877,0.7192
1,0.8762,0.9643,0.9701,0.5884,0.7325,0.6581,0.6929
2,0.8786,0.9691,0.9641,0.5946,0.7356,0.6624,0.6950
3,0.8821,0.9683,0.9721,0.6010,0.7428,0.6717,0.7044
4,0.8804,0.9691,0.9761,0.5968,0.7407,0.6688,0.7029
5,0.8849,0.9718,0.9900,0.6041,0.7504,0.6811,0.7162
6,0.8740,0.9689,0.9641,0.5847,0.7280,0.6523,0.6868
7,0.8754,0.9668,0.9820,0.5857,0.7338,0.6591,0.6966
8,0.8719,0.9661,0.9820,0.5788,0.7283,0.6517,0.6907


Original model was better than the blended model, hence it will be returned. NOTE: The display metrics are for the blended model (not the original one).


In [28]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Decision Tree Classifier,0.8791,0.9680,0.9730,0.5943,0.7379,0.6652,0.6993


In [29]:
result = pull()

#### Interpret the ensemble model

In [30]:
!pip install shap
interpret_model(top5_models[0], save=True)


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Plot the ensemble model

In [31]:
result.to_csv('models_blob2.csv')

#### Finalize and save the ensemble model

In [32]:
final_best = finalize_model(blended_models)
save_model(final_best, 'model_blob2')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['agreement_experts',
                                              'agreement_smell_type',
                                              'agreement_smell',
                                              'Organic_WeighOfClass',
                                              'Organic_NumberOfAccessorMethods',
                                              'Organic_ClassEffectiveLinesOfCode',
                                              'Organic_LCOM3',
                                              'Organic_TightClassCohesion',
                                              'Organic_PublicFieldCount',
                                              'Organic_Override...
                                     transformer=CleanColumnNames(match='[\\]\\[\\,\\{\\}\\"\\:]+'))),
                 ('actual_estimator',
             